<a href="https://colab.research.google.com/github/PPM5658/AskTheImage/blob/main/SQL_query_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install -qU transformers accelerate bitsandbytes torch

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
model_name='mistralai/Mistral-7B-Instruct-v0.2'
tokenizer=AutoTokenizer.from_pretrained(model_name)
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    load_in_8bit=True,
    torch_dtype=torch.float16
)

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [3]:
def build_prompt(schema, question):
    return f"""
You are an expert PostgreSQL developer.

Rules:
- Use ONLY the tables and columns from the schema
- Generate ONLY a SQL SELECT query
- Do NOT explain anything
- Do NOT add markdown
- Do NOT hallucinate columns

Schema:
{schema}

Question:
{question}

SQL:
"""


In [69]:
def clean_sql(text: str) -> str:
    return (
        text.replace("```sql", "")
            .replace("```", "")
            .replace("`", "")
            .replace("\n", " ")
            .strip()
    )

In [70]:
def generate_sql(question):
    schema = """
customers(id, name, city)
orders(id, customer_id, amount, order_date)
"""

    prompt = build_prompt(schema, question)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.2,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]

    sql = tokenizer.decode(
        generated_tokens,
        skip_special_tokens=True
    )

    return clean_sql(sql)

In [71]:
generate_sql('How many customers are from Berlin?')

"SELECT COUNT(DISTINCT customers.id) FROM customers WHERE city = 'Berlin';"

In [72]:
generate_sql('How many customers have purchased amount more than 1000 ?')

'SELECT customers.id, customers.name FROM customers JOIN orders ON customers.id = orders.customer_id WHERE orders.amount > 1000;'